In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


In [6]:
#csv 파일 읽기 
meta = pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")

#데이터 정제 (필요한 열만 추려내기)
meta = meta[ ['id', 'original_title', 'original_language', 'genres'] ]

#열 이름 변경
meta = meta.rename(columns = {'id' : 'movieId',
                             'original_title' : 'title',
                             'original_language' : 'language'})

#language가 'en'인 데이터만 솎아내기
meta = meta.loc[meta['language'] == 'en']
#, : 또는 지우면 해당 조건을 만족하는 row의 모든 col 가져옴

#movieId의 타입을 숫자로 변경하기 (object -> int)
meta.movieId = pd.to_numeric(meta.movieId)

#genre 정제하기: json string을 python set으로 변환하는 함수 구현
def str_to_set(x):
    myset = set()
    for d in eval(x):
        myset.add(d['name'])
    return myset

meta.genres = meta.genres.apply(str_to_set)
meta

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,movieId,title,language,genres
0,862,Toy Story,en,"{Animation, Family, Comedy}"
1,8844,Jumanji,en,"{Family, Fantasy, Adventure}"
2,15602,Grumpier Old Men,en,"{Comedy, Romance}"
3,31357,Waiting to Exhale,en,"{Drama, Romance, Comedy}"
4,11862,Father of the Bride Part II,en,{Comedy}
...,...,...,...,...
45459,222848,Caged Heat 3000,en,{Science Fiction}
45460,30840,Robin Hood,en,"{Drama, Action, Romance}"
45463,67758,Betrayal,en,"{Drama, Action, Thriller}"
45464,227506,Satana likuyushchiy,en,{}


In [10]:
#csv 파일 읽기
keywords = pd.read_csv("/kaggle/input/the-movies-dataset/keywords.csv")

#keywords 데이터의 keywords를 집합으로 변환하기
keywords.keywords = keywords.keywords.apply(str_to_set)
#keywords.keywords == keywords['keywords']

# id열을 movieId로 이름 변경하고 데이터 타입을 숫자로 변환하기
keywords = keywords.rename(columns = {'id' : 'movieId'})
#keywords.movieId = pd.to_numeric(keywords.movieId) 이미 int 64
#meta랑 타입이 맞지 않으면 해주어야 함

#같은 movieId를 갖는 데이터를 합치기
meta = pd.merge(meta, keywords, on='movieId', how='inner')
meta

#inner : 양쪽에 있는 것만 남겨둠 table A와 B의 교집합 행이 1, 2 라면 1, 2만
#outer : 하나라도 있으면 남김, 없는 쪽은 null로 채움
#left : 왼쪽 테이블에 있는 건 전부 남김 테이블의 크기가 무조건 왼쪽과 같아짐 
#right : 위와 같음

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Animation, Family, Comedy}","{boy, toy, toy comes to life, friends, friends..."
1,8844,Jumanji,en,"{Family, Fantasy, Adventure}","{giant insect, board game, disappearance, base..."
2,15602,Grumpier Old Men,en,"{Comedy, Romance}","{old men, fishing, duringcreditsstinger, best ..."
3,31357,Waiting to Exhale,en,"{Drama, Romance, Comedy}","{chick flick, interracial relationship, single..."
4,11862,Father of the Bride Part II,en,{Comedy},"{mother daughter relationship, confidence, mid..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Drama, Action, Romance}",{}
32849,67758,Betrayal,en,"{Drama, Action, Thriller}",{}
32850,227506,Satana likuyushchiy,en,{},{}


In [18]:
#Jaccard Similarity

#영화 찾아보고 합쳐서 출력 해보기
dk = meta.loc[meta['title'] == 'The Dark Knight'].iloc[0]
dkr = meta.loc[meta.title == 'The Dark Knight Rises'].iloc[0]

pd.concat([dk, dkr], axis = 1).T
#concat -> 합침, axis = 1 -> table 형태로, .T -> transpose

#Jaccard 유사도 함수 구현 및 실행
def jaccard_similarity(s1, s2):
    if len(s1|s2) == 0:
        return 0
    return len(s1&s2)/len(s1|s2)

jaccard_similarity(dk.genres|dk.keywords, dkr.genres|dkr.keywords)
#장르와 키워드 합쳐서 인자로 넘김

0.37142857142857144

In [29]:
#csv 파일 읽기
ratings = pd.read_csv("/kaggle/input/the-movies-dataset/ratings_small.csv")

#Rating의 movieId를 숫자로 변환
#이미 int 64 마찬가지로 meta와 타입이 맞지 않으면 해주기
ratings.movieId = pd.to_numeric(ratings.movieId)

#metadata로부터 title 정보 가져와 합치기
ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')
#movieId에 대해 merge, meta의 title 정보 가져옴

#pivot table 만들기
matrix = ratings.pivot_table(index='userId', columns='title', values='rating')
#index == row, columns == col

#Pearson 유사도 함수 구현
def pearson_similarity(u1, u2):
    u1_c = u1 - u1.mean() #모든 평점이 긍정적, 극대화 시키기 -> 평균 빼
    u2_c = u2 - u2.mean()
    
    denom = np.sqrt(np.sum(u1_c ** 2) * np.sum(u2_c ** 2))
    if denom != 0:
        return np.sum(u1_c * u2_c)/denom
    else:
        return 0
    
#The Dark Knight와 Prom Night의 Pearson 유사도 구해보기
dk_rating = matrix['The Dark Knight']
pk_rating = matrix['Prom Night']

pearson_similarity(dk_rating, pk_rating)

0.773565934694095

In [32]:
#비슷한 영화 추천 기능 구현
def find_similar_movies(input_title, matrix, n, alpha):
    input_meta = meta.loc[meta.title == input_title].iloc[0]
    #입력된 영화의 metadata
    input_set = input_meta.genres | input_meta.keywords
    #입력된 영화의 genres와 keywords의 합집합
    
    result = [] #모든 영화마다 유사도를 계산하여 저장하는 리스트
    
    for this_title in matrix.columns:
        if this_title == input_title: #입력된 영화는 유사도 검사 X
            continue
        
        this_meta = meta.loc[meta.title == this_title].iloc[0]
        #유사도를 계산하려는 영화의 matadata
        this_set = this_meta.genres | this_meta.keywords
        #이 영화의 genres와 keywords의 합집합
        
        pearson = pearson_similarity(matrix[this_title], matrix[input_title])
        #입력 영화와 이번 영화의 pearson 유사도 결과
        jaccard = jaccard_similarity(this_set, input_set)
        #입력 영화와 이번 영화의 jaccard 유사도 결과
        
        score = alpha * pearson + (1-alpha) * jaccard
        #유사도 검사 결과의 가중치 합
        result.append((this_title, pearson, jaccard, score))
        #계산 결과 추가
        
    result.sort(key=lambda r: r[3], reverse=True)
    #유사도 검사 후 결과 값 sorting
    
    return result[:n] #상위 n개 return

In [33]:
#test - The Dark Knight와 비슷한 영화 추천 해보기
test = find_similar_movies('The Dark Knight', matrix, 10, 0.3)
pd.DataFrame(test, columns = ['title', 'pearson', 'jaccard', 'score'])

,title,pearson,jaccard,score
0,Wild Wild West,0.773566,0.032258,0.254650
1,Prom Night,0.773566,0.022222,0.247625
2,Batman Begins,0.005015,0.292683,0.206383
3,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.200643
4,Blue Thunder,0.326617,0.133333,0.191318
5,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.189930
6,Topaz,0.377145,0.103448,0.185557
7,Big Bad Mama,0.344649,0.107143,0.178395
8,Sneakers,0.415830,0.068966,0.173025
9,The Enforcer,0.326617,0.103448,0.170399
